In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
import os
import random
import pandas as pd
from datasets import Dataset
from google.colab import drive
import gc
import torch
from datasets import concatenate_datasets

gc.collect()
torch.cuda.empty_cache()

drive.mount('/content/drive')

model_id = "Emilio407/guarani-jopara-llama-3.1-8B-instruct-v1" #"Emilio407/guarani-jopara-gemma-2-2b-it-v1"

# Lê JSON original
df = pd.read_json("/content/drive/MyDrive/Doutorado Unesp/assistente-guarani/data/dados_treinamento_guarani.json")

# Converte para Dataset Hugging Face
dataset = Dataset.from_pandas(df)

# Aumenta os dados
instructions_pt = [
    "Traduza para o português:",
    "Tradução em português:",
    "O que significa em português:",
    "Traduza do Guarani para o Português:",
    "Como se diz em português:",
    "Converta para o português:",
    "Passe para o português:",
    "A tradução portuguesa é:",
    "Explique em português:",
    "Diga isso em português:",
    "Interprete isso em português:",
    "Tradução correta em português:",
    "Versão em português:",
    "Tradução literal para o português:",
    "Tradução aproximada em português:",
    "Tradução com sentido cultural em português:"
]
instructions_gua = [
    "Traduza para o guarani:",
    "Versão em Guarani:",
    "O que significa em Guarani:",
    "Tradução em Guarani:",
    "Como se diz em Guarani:",
    "Com respeito à cultura Guarani",
    "Em guarani, significa:",
    "Explique em Guarani:",
    "Tradução para o Guarani tradicional:",
    "Tradução para o Guarani moderno:",
    "Tradução Jopara:",
    "Versão Guarani com contexto cultural:",
    "Diga isso em Guarani:",
    "Converta para Guarani:",
    "Traduzindo para Guarani com respeito:",
    "Tradução respeitosa em Guarani:"
]

augmented_data = []

for _, row in df.iterrows():
    for _ in range(20):
        inst = random.choice(instructions_pt)
        augmented_data.append({
            "instruction": inst,
            "input": row["input"],
            "output": row["output"]
        })
        inst_inv = random.choice(instructions_gua)
        augmented_data.append({
            "instruction": inst_inv,
            "input": row["output"],
            "output": row["input"]
        })

print(f"✅ Dataset aumentado: {len(augmented_data)} exemplos")

# Cria dataset a partir dos dados aumentados
df_augmented = pd.DataFrame(augmented_data)
dataset_augmented = Dataset.from_pandas(df_augmented)

# Junta os datasets original + aumentado
full_dataset = concatenate_datasets([dataset, dataset_augmented])

# Carrega tokenizer e modelo
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Dataset aumentado: 26720 exemplos


/usr/local/lib/python3.11/dist-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

...

pytorch_model-00001-of-00004.bin:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00004.bin:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

In [2]:
import gc
import torch
from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments

# ⚠️ Limpa cache ANTES da tokenização e treinamento
gc.collect()
torch.cuda.empty_cache()

# ⚙️ Tokenização mais leve
def tokenize(example):
    prompt = example["instruction"] + "\n" + example["input"] + "\n"
    target = example["output"]
    full_text = prompt + target

    tokenized = tokenizer(
        full_text,
        truncation=True,
        padding="max_length",  # pode trocar para "longest" se quiser evitar truncamentos desnecessários
        max_length=96  # 🔻 reduzido de 128 → 96 para economizar memória
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# 🔁 Tokeniza todo o dataset combinado (original + aumentado)
tokenized_dataset = full_dataset.map(tokenize, remove_columns=full_dataset.column_names)

# 🤖 Collator causal sem máscara
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # causal LM
)

# ⚙️ Parâmetros de treino otimizados
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Doutorado Unesp/assistente-guarani/data/modelo_finetunado",
    per_device_train_batch_size=1,   # 🔻 manter 1 para evitar OOM
    gradient_accumulation_steps=4,   # ⚡ Adicionado para acumular gradientes em 4 passos
    num_train_epochs=3,              # 🔻 opcionalmente reduzido para 3 para testes iniciais
    save_steps=500,
    save_total_limit=1,
    fp16=False,                       # 🔥 Ativa FP16 para economia de memória (se suportado)
    logging_dir="logs",
    logging_steps=100,
    report_to=[]  # evita dependências externas (como wandb)
)

# 🔧 Cria o trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# ♻️ Limpa memória antes de treinar
gc.collect()
torch.cuda.empty_cache()

# 🚀 Inicia o fine-tuning
trainer.train()

# 💾 Salva o modelo
output_dir = "/content/drive/MyDrive/Doutorado Unesp/assistente-guarani/data/modelo_finetunado"
trainer.save_model(output_dir)

Map:   0%|          | 0/27388 [00:00<?, ? examples/s]

/tmp/ipython-input-2-76664789.py:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 0 has a total capacity of 22.16 GiB of which 145.38 MiB is free. Process 10320 has 22.01 GiB memory in use. Of the allocated memory 21.83 GiB is allocated by PyTorch, and 1.25 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:

# === Teste do modelo treinado ===

# Recarrega para garantir tudo certo
tokenizer = AutoTokenizer.from_pretrained(output_dir)
model = AutoModelForCausalLM.from_pretrained(output_dir)
model.eval()

prompts = [
    "Como se diz 'gavião' em Guarani?",
    "Traduza para o guarani: A escola está aberta.",
    "Traduza para o guarani: Eu te amo.",
    "Traduza para o guarani: O sol está brilhando.",
    "Traduza para o guarani: Vamos à floresta.",
    "Traduza para o guarani: A criança está brincando.",
    "Traduza para o guarani: Minha casa é pequena, mas feliz.",
    "Traduza para o guarani: Estamos em paz.",
    "Explique em Guarani: amizade é confiança.",
    "Explique em Guarani: saúde vem da natureza.",
    "Explique em Guarani: aprender é crescer.",
    "Explique em Guarani: respeito aos mais velhos é sagrado.",
    "Explique em Guarani: a língua Guarani é parte de nossa identidade.",
    "Explique em Guarani: água é vida.",
    "Explique em Guarani: educação é o caminho para o futuro.",
    "Explique em Guarani: precisamos preservar a floresta.",
]

for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        inputs['input_ids'],
        max_length=30,
        do_sample=True,
        temperature=0.5,
        top_p=0.8,
        num_return_sequences=3
    )

    resposta_bruta = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    if resposta_bruta.startswith(prompt):
        resposta = resposta_bruta[len(prompt):].strip()
    else:
        resposta = resposta_bruta

    print(f"🗣️ Prompt: {prompt}")
    print(f"✅ Resposta: {resposta}\n")
